In [7]:
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OrdinalEncoder

In [9]:
#MLP CLASSIFIER  
housing = pd.read_csv("housing.csv")
housing.dropna(axis=0, inplace=True)

In [12]:
#Setting the training and test data sets
housing_X = housing.drop("ocean_proximity", axis=1) # drop labels 
#Select the label
housing_target = housing["ocean_proximity"]

In [13]:

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
housing_X_std = scaler.fit_transform(housing_X)


In [14]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(housing_X_std, housing_target, test_size=0.5)

In [15]:
#WITH OUT EARLY STOPPING
from sklearn.neural_network import MLPClassifier
dnn_clf = MLPClassifier(hidden_layer_sizes=(15,10,7),max_iter=500)
dnn_clf.fit(train_x, train_y)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(15, 10, 7), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [16]:
#Let's check the number of iterations:
dnn_clf.n_iter_

241

In [17]:
test_y_pred = dnn_clf.predict(test_x)
confusion_matrix(test_y, test_y_pred)

array([[4305,  120,    0,   11,  124],
       [  91, 3075,    0,    5,    0],
       [   0,    0,    0,    0,    4],
       [  17,    6,    0, 1130,    3],
       [ 316,    0,    0,   99,  911]])

In [18]:
print(accuracy_score(test_y, test_y_pred))

0.922090633258295


In [19]:
#WITH EARLY STOPPING
dnn_clf1 = MLPClassifier(hidden_layer_sizes=(15,10,7),max_iter=500,
                       early_stopping=True)
dnn_clf1.fit(train_x, train_y)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=True, epsilon=1e-08,
              hidden_layer_sizes=(15, 10, 7), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [20]:
#Let's check the number of iterations:
dnn_clf1.n_iter_

91

In [21]:
test_y_pred1 = dnn_clf1.predict(test_x)
confusion_matrix(test_y, test_y_pred)

array([[4305,  120,    0,   11,  124],
       [  91, 3075,    0,    5,    0],
       [   0,    0,    0,    0,    4],
       [  17,    6,    0, 1130,    3],
       [ 316,    0,    0,   99,  911]])

In [31]:
print(accuracy_score(test_y, test_y_pred1))

0.891846921797005


In [22]:
#Despite early stopping made low iterations(91 while without early stopping it was 241), 
#the accuracy decreased in this case with the early stopping.

In [23]:
#KERAS
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
# fix random seed for reproducibility
np.random.seed(42)

In [24]:
#PREPARING THE DATA
#Set the training and test data sets
housing_num = housing.drop("ocean_proximity", axis=1) # drop labels 
#Select the label
housing_target = housing[["ocean_proximity"]]
housing_num_std = scaler.fit_transform(housing_num)
ordinal_encoder = OrdinalEncoder()
housing_labels_ord = ordinal_encoder.fit_transform(housing_target)
housing_labels_int = housing_labels_ord.astype(int)
train_x, test_x, train_y, test_y = train_test_split(housing_num_std, housing_labels_int, test_size=0.5)

In [25]:
#Define the model: for multi-class
model = Sequential()
model.add(Dense(15, input_dim=9, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(7, activation='relu'))
#final layer: there has to be 5 nodes with softmax (because we have 5 categories)
model.add(Dense(5, activation='softmax'))

In [26]:
# Compile model
#Optimizer:
sgd = keras.optimizers.SGD(lr=0.05)
model.compile(loss='sparse_categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [27]:
model.fit(train_x, train_y, epochs=500)

Epoch 1/500
320/320 [==============================] - 1s 1ms/step - loss: 0.9583 - accuracy: 0.6252
Epoch 2/500
320/320 [==============================] - 0s 1ms/step - loss: 0.6347 - accuracy: 0.7609
Epoch 3/500
320/320 [==============================] - 0s 1ms/step - loss: 0.4999 - accuracy: 0.7987
Epoch 4/500
320/320 [==============================] - 0s 1ms/step - loss: 0.4297 - accuracy: 0.8243
Epoch 5/500
320/320 [==============================] - 0s 1ms/step - loss: 0.4026 - accuracy: 0.8367
Epoch 6/500
320/320 [==============================] - 0s 1ms/step - loss: 0.3865 - accuracy: 0.8407
Epoch 7/500
320/320 [==============================] - 0s 1ms/step - loss: 0.3645 - accuracy: 0.8527
Epoch 8/500
320/320 [==============================] - 0s 1ms/step - loss: 0.3556 - accuracy: 0.8565
Epoch 9/500
320/320 [==============================] - 0s 1ms/step - loss: 0.3517 - accuracy: 0.8576
Epoch 10/500
320/320 [==============================] - 0s 1ms/step - loss: 0.3342 - accura

In [28]:
test_y_pred3 = model.predict(test_x)

In [29]:
scores = model.evaluate(test_x, test_y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

320/320 [==============================] - 0s 1ms/step - loss: 0.3288 - accuracy: 0.8727

accuracy: 87.27%


In [ ]:
#WE GOT A BETTER ACCURACY USING KERAS